In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
import root_pandas as rpd
from root_pandas import read_root


mpl.rcParams.update({'font.size': 18})
mpl.rcParams.update({'legend.fontsize': 20})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})

In [ ]:
def getRatio(df_A,df_D,df_trigger_A,df_trigger_D, variable='z_asso',trig_cut = 'z>0.5', pair_cut='',minz=0.08,maxz=0.45,nbins=9, applyweight=False):
    #get number of pions with z>0.5
    print ('Print Trigger Cut ' ,trig_cut)
    print ('Total Cut ', trig_cut + pair_cut)
    
    norm_A = df_trigger_A.query(trig_cut).shape[0]
    norm_D = df_trigger_D.query(trig_cut).shape[0]
    
    #norm_A , xx = np.histogram( df_trigger_A.query(trig_cut),bins=np.array([0.5,1.0]))
    
    if(applyweight):
        y_A, x_conditional = np.histogram(df_A.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins),weights=df_A.query(trig_cut+pair_cut)['weight'])
        y_D, x_conditional = np.histogram(df_D.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins),weights=df_D.query(trig_cut+pair_cut)['weight'])
        erry_A, x_conditional = np.histogram(df_A.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins),weights=df_A.query(trig_cut+pair_cut)['weight2'])
        erry_D, x_conditional = np.histogram(df_D.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins),weights=df_D.query(trig_cut+pair_cut)['weight2'])
    
    else:
        y_A, x_conditional = np.histogram(df_A.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins))
        y_D, x_conditional = np.histogram(df_D.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins))
        erry_A, x_conditional = np.histogram(df_A.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins))
        erry_D, x_conditional = np.histogram(df_D.query(trig_cut+pair_cut)[variable],bins=np.logspace(np.log10(minz), np.log10(maxz),nbins))
  
    x_conditional = (x_conditional[1:] + x_conditional[:-1])/2.0
    err_A = np.true_divide(np.sqrt(erry_A),y_A)
    err_D = np.true_divide(np.sqrt(erry_D),y_D)
    y_A = np.true_divide(y_A,norm_A)
    y_D = np.true_divide(y_D,norm_D)
    ratio_conditional = np.true_divide(y_A,y_D)
    error_conditional = np.multiply(ratio_conditional, np.sqrt(np.power(err_A,2.0) + np.power(err_D,2.0)))
    
    return ratio_conditional,error_conditional,x_conditional

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    nbeforecut = inputDataframe.shape[0]
    cutDataframe = None
    if nbeforecut>0:
        cutDataframe = inputDataframe.query(cut)
        if text:
            print (text, cutDataframe.shape[0], ' (%2.2f '%(100.0*cutDataframe.shape[0]/nbeforecut), '%)')
    return cutDataframe
def applyCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    dataframe.eval('E = z*Nu', inplace=True)
    dataframe.eval('P = sqrt(E*E-0.140*0.140)', inplace=True)
    
    dataframe = applyCut(dataframe, 'Q2>1.0', 'Q2>1.0 :')
    #dataframe = applyCut(dataframe, 'Nu>3.0 and Nu<3.5', '3.0 < Nu < 3.5')
    dataframe = applyCut(dataframe, 'z>0.5', 'z>0.5 :')
    dataframe = applyCut(dataframe, 'P <2.5 ', 'P<2.5 ')

    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    return dataframe

def applyCutsPair(fullDataframe,name='default',isMC=False):
    print ('Starting election on dipion variables')
    if (isMC):
        print ('This is MC')
    else: 
        print ('This is Data')
    
    dataframe = fullDataframe
    dataframe.eval('z_tot = z+z_asso', inplace=True)
    dataframe.eval('E_asso = z_asso*Nu', inplace=True)
    dataframe.eval('P_asso = sqrt(E_asso*E_asso-0.140*0.140)', inplace=True)
    dataframe = applyCut(dataframe, 'Q2>1.0', 'Q2>1.0 :')
    #dataframe = applyCut(dataframe, 'Nu>3.0 and Nu<3.5', '3.0 < Nu < 3.5')
    dataframe = applyCut(dataframe, 'z>0.5', 'z>0.5 :')
    dataframe = applyCut(dataframe, 'P <2.5 ', 'P<2.5 ')

    dataframe = applyCut(dataframe, 'pid*pid_asso<0', 'Opposite sign pairs')
    
    ##Polar angle acceptance, different for 
    #if(isMC==False):
    #    print 'Polar angle acceptance'
    #    dataframe = applyCut(dataframe,'(pid_asso==211 & theta_lab_asso>10 & theta_lab_asso<90)|(pid_asso==-211 & theta_lab_asso>45 & theta_lab_asso<90)')
    if (not isMC):
        
        dataframe = applyCut(dataframe, 'theta_lab_asso<120 and theta_lab_asso>10', '10<theta_asso<120')
        dataframe = applyCut(dataframe, 'P_asso>0.200', 'P_asso>0.200')
        dataframe = applyCut(dataframe,'(pid_asso==211)| (pid_asso==-211 & theta_lab_asso>30)| (pid_asso==-211 & theta_lab_asso<30 & P_asso>0.500)','P>500 for pi- with theta<30')

        #dataframe = applyCut(dataframe,'(pid_asso==211 & theta_lab_asso>10.0)|(pid_asso==-211 & theta_lab_asso>30 & theta_lab_asso<120)')
        #dataframe = applyCut(dataframe, '(pid_asso==211 & P_asso>0.200) |(pid_asso==-211 & P_asso>0.300) ', 'pi+ P_asso > 200 MeV, pi- P_asso > 300 MeV')
    dataframe = applyCut(dataframe, 'P_asso<2.5', 'P_asso <2.5 GeV')
    #dataframe = applyCut(dataframe, 'mass<1.4', 'mass < 1.4 GeV')
    
    return dataframe

def printPairBreakdown(dataframe):
    allpairs = 1.0*dataframe.shape[0]
    print ('All pairs ', allpairs)
    print ('Pairs with Leading pi+', np.true_divide(dataframe.query('pid==211').shape[0],allpairs))
    print ('Pairs with Leading pi-', dataframe.query('pid==-211').shape[0]/allpairs)
    print ('Pairs with Sub-Leading pi+', dataframe.query('pid_asso==211').shape[0]/allpairs)
    print ('Pairs with Sub-Leading pi-', dataframe.query('pid_asso==-211').shape[0]/allpairs)
    print ('pi+ pi+ pairs',dataframe.query('pid==211 and pid_asso==211').shape[0]/allpairs)
    print ('pi- pi- pairs',dataframe.query('pid==-211 and pid_asso==-211').shape[0]/allpairs)
    print ('pi+ pi- pairs',dataframe.query('pid==211 and pid_asso==-211').shape[0]/allpairs)
    print ('pi- pi+ pairs',dataframe.query('pid==-211 and pid_asso==211').shape[0]/allpairs)
    print ('//////////////////////////////////////////////////////')
    return 


In [ ]:
def getAcceptance(momentum=1.0,theta=20.0, pid=211, targtype='solid'):
    x = np.array([0.0325, 0.0975 , 0.1625 , 0.2275 , 0.2925 , 0.3575 , 0.4225 , 0.4875 , 0.5525 , 0.6175 , 0.6825 , 0.7475 , 0.8125 , 0.8775 , 0.9425 , 1.0075 , 1.0725 , 1.1375 , 1.2025 , 1.2675  ])  
    solidtarget_pim = {}
    liquidtarget_pim = {}
    
        
    solidtarget_pim['1'] = np.array([0,0 ,0 ,0 ,0 ,3.23167e-06 ,0.001560151 , 0.01057529,0.01825397 ,0.0261568 , 0.03116873,0.03466488 , 0.03863429,0.04103578 , 0.04319868,0.04477068 ,0.0459576 ,0.04610074 ,0.04597379 ,0.041296720])
    liquidtarget_pim['1'] = np.array([ 0 ,0 ,0 ,0 ,0 ,1.61536e-06 ,0.002657303 ,0.01237076 , 0.02036797,0.02867371 ,0.03345114 ,0.0380118 , 0.04140325, 0.04386796, 0.04682532,0.04792033 ,0.04949423 ,0.04856436 ,0.04892293 ,0.04327102 ])

    solidtarget_pim['2'] = np.array([ 0 ,0 ,0.002629189 ,0.03186881 ,0.06729287 ,0.1043241 , 0.1307755, 0.1355177, 0.1339914, 0.1398696,0.1373173 ,0.1340744 ,0.1331808 , 0.130854, 0.1272746, 0.1236089,0.1165032 ,0.1128401 ,0.1072237 ,0.09178725  ])
    liquidtarget_pim['2'] = np.array([0. ,0. ,0.002954789 ,0.03605334 ,0.07328045 ,0.1112898 ,0.1340548 ,0.1370217 ,0.1376773 ,0.1419643 ,0.1419834 ,0.1377389 ,0.1373697 ,0.1353414 ,0.132038 ,0.1265082 ,0.1199043 ,0.1178251 ,0.1116354 ,0.09549689  ])

    solidtarget_pim['3'] = np.array([ 0.0002231072 ,0.1211735 ,0.2128658 ,0.2396404 ,0.2553216 ,0.2658868 ,0.2700335 ,0.2717077 ,0.2731176 ,0.2821284 ,0.2848298 ,0.2837925 ,0.2846656 ,0.2829416 ,0.2791555 ,0.2721919 ,0.2692559 ,0.2690514 ,0.2679086 ,0.2478978 ])
    liquidtarget_pim['3'] = np.array([ 0.00660692 ,0.1578462 ,0.2173256 ,0.2422208 ,0.257946 ,0.2675358 ,0.2723963 ,0.2755941 ,0.2744294 ,0.2831314 ,0.2860147 ,0.2841786 ,0.2849058 ,0.2853593 ,0.2794619 ,0.2754566 ,0.2712428 ,0.2708606 ,0.2692244 ,0.2487378 ])

    solidtarget_pim['4'] = np.array([ 0.001333508,0.1449459 ,0.2186205 ,0.2415341 ,0.2531107 ,0.2574371 ,0.2609193 ,0.2617711 ,0.262772 ,0.2752933 ,0.2799949 ,0.2802792 ,0.281052 ,0.2796929 ,0.2742227 ,0.2711661 ,0.2740372 ,0.2708804 ,0.2731924 ,0.2612534 ])
    liquidtarget_pim['4'] = np.array([ 0.02327899 ,0.1629743 ,0.2106435 ,0.2306853 ,0.2454332 ,0.2509662 ,0.2547484 ,0.256513 ,0.2575279 ,0.2676785 ,0.2702532 ,0.2706461 ,0.2732721 ,0.2716608 ,0.2654282 ,0.2629694 ,0.2660801 ,0.263469 ,0.2604181 ,0.2622458 ])

    solidtarget_pim['5'] = np.array([0.006982775 ,0.1447486 ,0.1897953 ,0.2084306 ,0.2172206 ,0.2223349 ,0.2240443 ,0.2250739 ,0.2231615 ,0.2327872 ,0.2361698 ,0.2290048,0.231793 ,0.2323416 ,0.2202763 ,0.2219534 ,0.2157027 ,0.2203591 ,0.2019774 ,0.1933508 ])
    liquidtarget_pim['5'] = np.array([0.03545699 ,0.1620397 ,0.1961021 ,0.2146192 ,0.2243374,0.2282763 ,0.2288964 ,0.2295778 ,0.2266956 ,0.2375714 ,0.2389445 ,0.2341746 ,0.2357495 ,0.2359444 ,0.224392 ,0.2227679 ,0.2211828 ,0.2209665 ,0.2165821 ,0.1870061 ])

    solidtarget_pim['6'] = np.array([0.009253875 ,0.1392653 ,0.1971637 ,0.2154679 ,0.2187641 ,0.225096,0.2284017 ,0.2305547 ,0.2317961 ,0.2408562 ,0.2482083 ,0.2436806 ,0.2344238 ,0.2462491 ,0.2138468 ,0.2307692 ,0.2372414 ,0.2157895 ,0.2 ,0. ])
    liquidtarget_pim['6'] = np.array([ 0.02504874 ,0.1437731 ,0.1910572 ,0.2113901 ,0.2182047 ,0.2246697 ,0.2270944 ,0.231018 ,0.2354003 ,0.2440679 ,0.2582594 ,0.2499597 ,0.2501701 ,0.2524341 ,0.2451745 ,0.2437037 ,0.2410468 ,0.2362205 ,0.16 ,0. ])

    solidtarget_pim['7'] = np.array([ 6.89698e-06 ,0.01220896 ,0.03071677 ,0.03531329 ,0.04548552 ,0.06199558 ,0.09144737 ,0.1081081,0.2222222 ,3 ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ])
    liquidtarget_pim['7'] = np.array([ 2.068238e-05 ,0.01000872 ,0.02016303 ,0.02335588 ,0.03246062 ,0.04609146 ,0.07527773 ,0.1166181 ,0.2666667 ,3.666667 ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ])

    
    solidtarget_pip = {}
    liquidtarget_pip = {}
    
    #//pi+
    solidtarget_pip['1'] = np.array([ 0. ,0.0263211 ,0.1196485 ,0.1645991 ,0.1794361 ,0.207905 ,0.2125451 ,0.2075122 ,0.2127852 ,0.2212154 ,0.2248536 ,0.226564 ,0.2281049 ,0.2298948 ,0.2274279 ,0.2259013 ,0.2228642 ,0.223829 ,0.2221386 ,0.1908589 ])
    liquidtarget_pip['1'] = np.array([0. ,0.03226664 ,0.1235518 ,0.1673732 ,0.1845819 ,0.2111029 ,0.2143435 ,0.2117669 ,0.2165755 ,0.2239562 ,0.2277253 ,0.2289291 ,0.2305655 ,0.2331748 ,0.2294087 ,0.2276427 ,0.2249865 ,0.2261938 ,0.2235695 ,0.1942732 ])

    solidtarget_pip['2'] = np.array([1.746534e-05 ,0.07758519 ,0.1789028 ,0.2142687 ,0.2309015 ,0.230986 ,0.231743,0.2343798 ,0.2348231 ,0.2442682 ,0.246481 ,0.2453539 ,0.2416622 ,0.2401279 ,0.2368902 ,0.2353596 ,0.2374409 ,0.2360786 ,0.2338877 ,0.221718 ])
    liquidtarget_pip['2'] = np.array([ 4.36685e-05,0.1020439 ,0.1824278 ,0.2151706 ,0.2314508 ,0.2324465 ,0.2368424 ,0.2384085 ,0.2403754 ,0.249458 ,0.2504815 ,0.247795 ,0.2451 ,0.2430814 ,0.2397604 ,0.2377985 ,0.2396885 ,0.2378236 ,0.2350796 ,0.2228359 ])

    solidtarget_pip['3'] = np.array([ 0.0002231072 ,0.1211735 ,0.2128658 ,0.2396404 ,0.2553216 ,0.2658868 ,0.2700335 ,0.2717077 ,0.2731176 ,0.2821284 ,0.2848298 ,0.2837925 ,0.2846656 ,0.2829416 ,0.2791555 ,0.2721919 ,0.2692559 ,0.2690514 ,0.2679086 ,0.2478978 ])
    liquidtarget_pip['3'] = np.array([ 0.00660692 ,0.1578462 ,0.2173256 ,0.2422208 ,0.257946 ,0.2675358 ,0.2723963 ,0.2755941 ,0.2744294 ,0.2831314 ,0.2860147 ,0.2841786 ,0.2849058 ,0.2853593 ,0.2794619 ,0.2754566 ,0.2712428 ,0.2708606 ,0.2692244 ,0.2487378 ])

    solidtarget_pip['4'] = np.array([0.001333508,0.1449459 ,0.2186205 ,0.2415341 ,0.2531107 ,0.2574371 ,0.2609193 ,0.2617711 ,0.262772 ,0.2752933 ,0.2799949 ,0.2802792 ,0.281052 ,0.2796929 ,0.2742227 ,0.2711661 ,0.2740372 ,0.2708804 ,0.2731924 ,0.2612534 ])
    liquidtarget_pip['4'] = np.array([  0.02327899 ,0.1629743 ,0.2106435 ,0.2306853 ,0.2454332 ,0.2509662 ,0.2547484 ,0.256513 ,0.2575279 ,0.2676785 ,0.2702532 ,0.2706461 ,0.2732721 ,0.2716608 ,0.2654282 ,0.2629694 ,0.2660801 ,0.263469 ,0.2604181 ,0.2622458 ])

    solidtarget_pip['5'] = np.array([ 0.006982775 ,0.1447486 ,0.1897953 ,0.2084306 ,0.2172206 ,0.2223349 ,0.2240443 ,0.2250739 ,0.2231615 ,0.2327872 ,0.2361698 ,0.2290048,0.231793 ,0.2323416 ,0.2202763 ,0.2219534 ,0.2157027 ,0.2203591 ,0.2019774 ,0.1933508 ])
    liquidtarget_pip['5'] = np.array([ 0.03545699 ,0.1620397 ,0.1961021 ,0.2146192 ,0.2243374,0.2282763 ,0.2288964 ,0.2295778 ,0.2266956 ,0.2375714 ,0.2389445 ,0.2341746 ,0.2357495 ,0.2359444 ,0.224392 ,0.2227679 ,0.2211828 ,0.2209665 ,0.2165821 ,0.1870061 ])

    solidtarget_pip['6'] = np.array([0.009253875 ,0.1392653 ,0.1971637 ,0.2154679 ,0.2187641 ,0.225096,0.2284017 ,0.2305547 ,0.2317961 ,0.2408562 ,0.2482083 ,0.2436806 ,0.2344238 ,0.2462491 ,0.2138468 ,0.2307692 ,0.2372414 ,0.2157895 ,0.2 ,0. ])
    liquidtarget_pip['6'] = np.array([ 0.02504874 ,0.1437731 ,0.1910572 ,0.2113901 ,0.2182047 ,0.2246697 ,0.2270944 ,0.231018 ,0.2354003 ,0.2440679 ,0.2582594 ,0.2499597 ,0.2501701 ,0.2524341 ,0.2451745 ,0.2437037 ,0.2410468 ,0.2362205 ,0.16 ,0. ])

    solidtarget_pip['7'] = np.array([ 6.89698e-06 ,0.01220896 ,0.03071677 ,0.03531329 ,0.04548552 ,0.06199558 ,0.09144737 ,0.1081081,0.2222222 ,3 ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ]);
    liquidtarget_pip['7'] = np.array([2.068238e-05 ,0.01000872 ,0.02016303 ,0.02335588 ,0.03246062 ,0.04609146 ,0.07527773 ,0.1166181 ,0.2666667 ,3.666667 ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ,0. ]);


    bin_number = None
    #print (theta)
    if(10 <theta<=30):
        bin_number = '1'
    elif(30<theta<=45):
        bin_number = '2'
    elif(45<theta<=60):
        bin_number = '3'
    elif(60<theta<=75):
        bin_number = '4'   
    elif(75<theta<=90):
        bin_number = '5'   
    elif(90<theta<=120):
        bin_number = '6'   
    elif(120<theta<=145):
        bin_number = '7' 
    else:
        print ('No valid theta value was passed=%2.2f'%theta)
    
    #print (bin_number)
    acc = 1.00
    
    idx = (np.abs(x-momentum)).argmin()
    #print (idx)
    
    
    if bin_number==None:
        acc = 1.0
    elif(targtype=='solid' and pid==-211):
        acc = solidtarget_pim[bin_number][idx]
    elif(targtype=='liquid' and pid==-211):
        acc = liquidtarget_pim[bin_number][idx]
    elif(targtype=='solid' and pid==211):
        acc = solidtarget_pip[bin_number][idx]
    elif(targtype=='liquid' and pid==211):
        acc = liquidtarget_pip[bin_number][idx]
    else:
        acc = 1.0
    #return momentum
    
    
    if(acc>0.0):
        weight = 1.0/acc
    else:
        weight = 1.0
        print('weight invalid')
        
    return weight





In [ ]:
def addAcceptanceInfo(df):
     
    #dataframe = fullDataframe
    df.eval('acc = @getAcceptance(P_asso)',inplace=True)
    return df


## Get data from Ntuples to dataframes

In [ ]:
#this needs to be read in chunks otherwise it uses all memory. (from https://github.com/scikit-hep/root_pandas)
def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    for df in read_root(filename, treename, chunksize=100000) :
        #print df.shape[0]
        dataframe = pd.concat([dataframe,df])
    
    print (dataframe.shape[0])
    return dataframe
    

## Get data

In [ ]:
df = {}
df_trigger = {}

for target in ['C','Fe','Pb']:
    print ('Getting dataframes from target %s'%target)
    df[target]=getdatainChunks('Pairs_%s.root'%target, target)
    df['D_%s'%target]= getdatainChunks('Pairs_%s.root'%target, 'D_%s'%target)  
    print ('Get trigger dataframes from deuterium target %s'%target)
    df_trigger['%s_trigger'%target] = getdatainChunks('Pairs_%s.root'%target, '%s_trigger'%target)
    df_trigger['D_%s_trigger'%target] = getdatainChunks('Pairs_%s.root'%target, 'D_%s_trigger'%target)

## GiBUU 

In [ ]:
for target in ['D','C','Fe','Pb']:
    #GiBUU
    print (target)
    print (' trigger')
    df_trigger['GiBUU_%s_trigger'%target]= getdatainChunks('GiBUU_Pairs_%s.root'%target, '%s_trigger'%target)
    print (' pairs')
    df['GiBUU_%s'%target]= getdatainChunks('GiBUU_Pairs_%s.root'%target, target)
print (df.keys())

## Combine the deuterium datasets 

In [ ]:
df['D'] = pd.concat([df['D_Pb'],df['D_Fe'],df['D_C']]) #simply combine the deuterium results for all targets
df_trigger['D_trigger'] = pd.concat([df_trigger['D_Pb_trigger'],df_trigger['D_Fe_trigger'],df_trigger['D_C_trigger']]) #simply combine the deuterium results for all targets



## Apply selection for pair ntuples

In [ ]:
print (df.keys())

In [ ]:
df['D'].hist(figsize=(12,12),bins=100,column=['dphi','dphi_lab','phi_pq','phi_lab'])
plt.show()

In [ ]:
#dataframees with pairs
for key in df.keys():
    isMC = False
    if 'GiBUU' in key:
        isMC=True
    print (key)
    
    df[key] = applyCuts(df[key],isMC=isMC)
    #printPairBreakdown(df[key])
    df[key] = applyCutsPair(df[key],isMC=isMC)

## Add weights to the dataframe

In [ ]:
for target in ['C','Fe','Pb']:
    df['%s'%target]['weight'] = df['%s'%target].apply(lambda x: getAcceptance(x['P_asso'], x['theta_lab_asso'],x['pid_asso'],'solid'), axis=1)
    df['D_%s'%target]['weight'] = df['D_%s'%target].apply(lambda x: getAcceptance(x['P_asso'], x['theta_lab_asso'],x['pid_asso'],'liquid'), axis=1)
    
    df['%s'%target]['weight2'] = df['%s'%target]['weight'].apply(lambda x: x*x)  
    df['D_%s'%target]['weight2'] = df['D_%s'%target]['weight'].apply(lambda x: x*x)  


In [ ]:
for key in df_trigger.keys():
    print (key)
    df_trigger[key] = applyCuts(df_trigger[key])
    print (' ') 

## R_2h data from HERMES and Neutrino buble-chamber experiment

In [ ]:
#hermes data:
hermes_y = {}
hermes_ystat = {}
hermes_ysyst = {}
hermes_ytotal = {}

#DATA FROM SELECTIO1 OF http://www-hermes.desy.de/notes/pub/publications.html
hermes_x = {}
hermes_x['Kr'] = [0.09,0.15,0.24,0.35,0.44]
hermes_x['N'] = np.subtract(hermes_x['Kr'],0.004)
hermes_x['Xe'] = np.subtract(hermes_x['Kr'],-0.004)

#nitrogen, atomic number 7
hermes_y['N']     = [1.0324, 0.9781,0.9293,0.8678,0.8822]
hermes_ystat['N'] = [0.0767,0.0268,0.0323,0.0478,0.0601]
hermes_ysyst['N'] = [0.0206,0.0196,0.0186,0.0174,0.0176]
hermes_ytotal['N'] = np.sqrt(np.power(hermes_ystat['N'],2.0)+np.power(hermes_ysyst['N'],2.0))

#krypton, atomic number 36
hermes_y['Kr'] = [1.2072,0.9180,0.8822,0.8631,1.0314]
hermes_ystat['Kr'] = [0.1342,0.0366,0.0452,0.0661,0.0972]
hermes_ysyst['Kr'] =[0.0241,0.0184,0.0176,0.0173,0.0206]
hermes_ytotal['Kr'] = np.sqrt(np.power(hermes_ystat['Kr'],2.0)+np.power(hermes_ysyst['Kr'],2.0))

hermes_y['Xe'] = [1.1648,0.9345,0.9242,0.8128,0.9373]
hermes_ystat['Xe'] = [0.1205,0.0359,0.0433,0.0614,0.0808]
hermes_ysyst['Xe'] = [0.0233,0.0187,0.0185,0.0163,0.0187]
hermes_ytotal['Xe'] = np.sqrt(np.power(hermes_ystat['Xe'],2.0)+np.power(hermes_ysyst['Xe'],2.0))


neutrino_x = [0.10763636363636367, 0.18618181818181817,0.25163636363636366,0.3345454545454546]
neutrino_y = [1.4535664335664338, 1.0148378893833438,  1.0373426573426574, 0.8784361093452003]
neutrino_yerror =[1.5794405594405596, 1.112739987285442, 1.1422377622377624,1.004310235219326]
neutrino_yerror = np.subtract(neutrino_yerror,neutrino_y)

fig = plt.figure(figsize=(8,6))



plt.errorbar(hermes_x['N'], hermes_y['N'],yerr=hermes_ytotal['N'],label='Hermes, $^{14}N$',fmt='o')
plt.errorbar(hermes_x['Kr'], hermes_y['Kr'],yerr=hermes_ytotal['Kr'],label='Hermes, $^{84}Kr$',fmt='o')
plt.errorbar(hermes_x['Xe'], hermes_y['Xe'],yerr=hermes_ytotal['Xe'],label='Hermes, $^{131}Xe$',fmt='o')
plt.errorbar(neutrino_x, neutrino_y,yerr=neutrino_yerror,label='SKAT, $A_{eff}=21$',fmt='o')
plt.xlabel('$z_{2}$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--',alpha=0.5)

plt.tight_layout()
plt.legend(frameon=False)
plt.savefig('PreviousData.png')
plt.savefig('PreviousData_R2h.pdf')

plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], 
                        df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                        minz=0.08)
    plt.errorbar(x,r,yerr=err,label='This work,  %s'%target,fmt='o',ms=8)

plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

plt.errorbar(hermes_x['N'], hermes_y['N'],yerr=hermes_ytotal['N'],fmt='o',mfc='white',label='HERMES, $^{14}_{7}N$',fillstyle=None,ms=8,capsize=5)
plt.errorbar(hermes_x['Kr'], hermes_y['Kr'],yerr=hermes_ytotal['Kr'],fmt='o',mfc='white',label='HERMES, $^{84}_{36}Kr$',fillstyle=None,ms=8,capsize=5)
plt.errorbar(hermes_x['Xe'], hermes_y['Xe'],yerr=hermes_ytotal['Xe'],fmt='o',mfc='white',label='HERMES, $^{131}_{54}Xe$',fillstyle=None,ms=8,capsize=5)
plt.errorbar(neutrino_x, neutrino_y,yerr=neutrino_yerror,fmt='s',mfc='white',label='Neutrino, $A_{eff}=21$',fillstyle=None,ms=8,capsize=5)

#plt.ylim([0.5,1.5])
#plt.xlim([0.00,0.5])
plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=2)

plt.tight_layout()
plt.savefig('ComparisonWithHermes_Linear.png')
#plt.xlim([0.05,0.5])

#plt.xscale('log')
#plt.savefig('ComparisonWithHermes_Log.png')

plt.show()

## Acceptance corrections

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], 
                        df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                        minz=0.08, applyweight=False)
    plt.errorbar(x,r,yerr=err,label='non weighted,  %s'%target,fmt='o',ms=8)
    
    
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], 
                        df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                        minz=0.08, applyweight=True)
    plt.errorbar(x+0.01,r,yerr=err,label='weighted,  %s'%target,fmt='o',ms=8)


plt.legend()
#plt.ylim([0.5,1.0])
plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

## Theory only

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'])
    plt.errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=8,lw=2)

plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)

plt.xlim([0.06,0.5])
plt.ylim([0.0,2.0])

plt.axhline(y=1.0,color='black',linestyle='--')
plt.legend(loc='best',frameon=False,fontsize=15,ncol=1)


plt.xscale('log')

plt.tight_layout()
plt.savefig('TheoryOnly.png')

## Result compared with GiBUU

In [ ]:
fig = plt.figure(figsize=(8,6))

colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

for i,target in enumerate(['C','Fe','Pb']):
    #data
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='z_asso',minz=0.07,maxz=0.5,applyweight=True
                       )
    plt.errorbar(np.subtract(x,0*0.0025),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])

    #gibuu
    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='z_asso',minz=0.07,maxz=0.5#,pair_cut='and z_asso>0.1'
                       )
    #plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.9,lw=5)
    plt.plot(x,r,label='GiBUU %s'%target,alpha=0.79,lw=3,color=colors[target])

#plt.xscale('log')
plt.xlabel('$z_2$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')
plt.ylim([0.0,1.5])
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2.png')


plt.errorbar(hermes_x['N'], hermes_y['N'],yerr=hermes_ytotal['N'],fmt='o',label='Hermes, N',fillstyle=None,ms=8,capsize=5,color='grey')
plt.errorbar(hermes_x['Kr'], hermes_y['Kr'],yerr=hermes_ytotal['Kr'],fmt='o',label='Hermes, Kr',fillstyle=None,ms=8,capsize=5,color='brown')
plt.errorbar(hermes_x['Xe'], hermes_y['Xe'],yerr=hermes_ytotal['Xe'],fmt='o',label='Hermes, Xe',fillstyle=None,ms=8,capsize=5,color='darkviolet')
#plt.errorbar(neutrino_x, neutrino_y,yerr=neutrino_yerror,fmt='s',mfc='white',label='Neutrino, $A_{eff}=21$',fillstyle=None,ms=8,capsize=5)

plt.tick_params(axis="x", labelsize=16)
plt.tick_params(axis="y", labelsize=16)
plt.text(0.20, 1.2, 'CLAS \nINTERNAL', fontsize=30)

plt.legend(frameon=False,ncol=3,fontsize=15)
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2_withHERMES.png')
plt.savefig('MainResult_withGiBUU_z2_withHERMES.pdf')


In [ ]:
fig = plt.figure(figsize=(8,6))

colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

for i,target in enumerate(['C','Fe','Pb']):
    #data
    
    
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='z_asso',minz=0.07,maxz=0.5,applyweight=True
                       )
    plt.errorbar(np.subtract(x,0*0.0025),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])

    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='z_asso',minz=0.07,maxz=0.5,applyweight=False
                       )
    plt.errorbar(np.subtract(x,0*0.0025),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])

   
plt.xlabel('$z_2$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')
#plt.ylim([0.0,1.5])
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2.png')


plt.tick_params(axis="x", labelsize=16)
plt.tick_params(axis="y", labelsize=16)

plt.legend(frameon=False,ncol=3,fontsize=15)
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2_withHERMES.png')
plt.savefig('MainResult_withGiBUU_z2_withHERMES.pdf')


In [ ]:
x_data = np.array([0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95])
y_data = {}
erry_data  ={}

y_data['pip_C'] = np.array([1.16751 , 1.01848 , 0.926135 , 0.876012 , 0.8595 , 0.848906 , 0.832911 , 0.762776 , 0.782656 , 0.698996 ])
erry_data['pip_C'] = np.array([0.00321583 , 0.00161351 , 0.00188183 , 0.00233672 , 0.00297862 , 0.00369811 , 0.00445801 , 0.00520291 , 0.00703654 , 0.00844543])
y_data['pip_Fe'] = np.array([1.18002 , 0.931569 , 0.772485 , 0.695442 , 0.653029 , 0.636352 , 0.609999 , 0.549825 , 0.538038 , 0.440439])
erry_data['pip_Fe'] = np.array([0.00297786 , 0.00136259 , 0.00146705 , 0.00175402 , 0.00214668 , 0.00261719 , 0.00307546 , 0.00352656 , 0.00457231 , 0.00516023])
y_data['pip_Pb'] = np.array([0.980339 , 0.747191 , 0.597805 , 0.522714 , 0.4807 , 0.466481 , 0.446264 , 0.399832 , 0.386629 , 0.277573 ])
erry_data['pip_Pb'] = np.array([0.0028235 , 0.00128281 , 0.00136618 , 0.00163387 , 0.00199655 , 0.0024583 , 0.00293294 , 0.00343855 , 0.00452875 , 0.0049117])



fig = plt.figure(figsize=(8,6))

colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

for i,target in enumerate(['C','Fe','Pb']):
    #data
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='z_asso',minz=0.07,maxz=0.5,applyweight=True
                       )
    plt.errorbar(np.subtract(x,0*0.0025),r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])
    plt.errorbar(x_data,y_data['pip_%s'%target],yerr=erry_data['pip_%s'%target],color=colors[target],fmt='s-',label=target)

    #gibuu
    #r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
    #                   ,variable='z_asso',minz=0.07,maxz=0.5#,pair_cut='and z_asso>0.1'
    #                   )
    #plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.9,lw=5)
    #plt.plot(x,r,label='GiBUU %s'%target,alpha=0.79,lw=3,color=colors[target])

    plt
#plt.xscale('log')
plt.xlabel('$z_2$',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')
#plt.ylim([0.0,1.5])
plt.xlim([0.0,0.5])

plt.tight_layout()
plt.savefig('MainResult_withGiBUU_z2.png')


In [ ]:
fig = plt.figure(figsize=(16,12))


label = {} 
label['C'] = '$^{12}C$'
label['Fe'] = '$^{56}Fe$'
label['Pb'] = '$^{208}Pb$'


colors = {}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'

colors['C_pip'] = '#ff7f0e'
colors['Fe_pip'] = '#d62728'
colors['Pb_pip']=  '#8c564b'


fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,6),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):

    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='z_asso',minz=0.07,maxz=0.5,applyweight=True
                       )
    axs[i].errorbar(np.subtract(x,0*0.0025),r,yerr=err,label='conditional, %s'%target,ms=8,fmt='o',alpha=0.8,color=colors[target])
    axs[i].fill_between(x, 1.03*r,0.97*r,alpha=0.2,color=colors[target])
   


    axs[i].set_xlim([0.0,0.5])

    axs[i].errorbar(x_data,y_data['pip_%s'%target],yerr=erry_data['pip_%s'%target],color='black',fmt='s',ms=8,alpha=0.7,label='inclusive')
    axs[i].fill_between(x_data, 1.03*y_data['pip_%s'%target],0.97*y_data['pip_%s'%target],color='black',alpha=0.2)

    
    axs[i].legend(loc='lower left',ncol=1,frameon=False,fontsize=19)
    axs[i].xaxis.set_label_text('$z$',fontsize=22)

axs[0].set_title('$^{12}C$',fontsize=22)
axs[1].set_title('$^{56}Fe$',fontsize=22)
axs[2].set_title('$^{208}Pb$',fontsize=22)
axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)


## Results vs GiBUU (mass)

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='mass',minz=0.4,maxz=1.8,nbins=12,pair_cut='and z_asso>0.15',applyweight=True
                       )
    plt.errorbar(x,r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8)

    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='mass',minz=0.4,maxz=1.8,nbins=12,pair_cut='and z_asso>0.15'
                       )
    plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)

plt.tick_params(axis="x", labelsize=16)
plt.tick_params(axis="y", labelsize=16)
plt.legend(frameon=False,ncol=2)
plt.xlabel('Dipion mass (GeV)',fontsize=22)
plt.ylabel('$R_{2h}$',fontsize=22)
plt.axhline(y=1.0,color='black',linestyle='--')

plt.text(0.60, 1.1, 'CLAS \nINTERNAL', fontsize=30)

#plt.ylim([0.0,1.2])
plt.tight_layout()
plt.savefig('MainResult_withGiBUU_mass.png')
plt.savefig('MainResult_withGiBUU_mass.pdf')

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='dipion_pt',minz=0.05,maxz=1.8,nbins=12,pair_cut='and z_asso>0.15'
                       )
    plt.errorbar(x,r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8)

    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='dipion_pt',minz=0.0,maxz=1.8,nbins=12,pair_cut='and z_asso>0.15'
                       )
    plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)


plt.legend(frameon=False)
plt.xlabel('Dipion transverse momentum (GeV)',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
plt.axhline(y=1.0,color='black',linestyle='--')

plt.tight_layout()
plt.savefig('MainResult_withGiBUU_mass.png')

## Results vs GiBUU (momentum P2)

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='P_asso',minz=0.200,maxz=1.0,nbins=7,applyweight=True
                       )
    plt.errorbar(x,r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8)

    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='P_asso',minz=0.200,maxz=1.0,nbins=7
                       )
    plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)

#plt.xscale('log')
plt.legend(frameon=False)
plt.xlabel('$P_{2}$ (GeV)',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
plt.axhline(y=1.0,color='black',linestyle='--')

plt.tight_layout()
plt.savefig('GiBUU_Momentum.png')

## Results vs GiBUU (ztot)

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='z_tot',minz=0.56,maxz=1.0,nbins=15
                       )
    plt.errorbar(x,r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8)

    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='z_tot',minz=0.56,maxz=1.0,nbins=15
                       )
    plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)

plt.xscale('log')
plt.legend(frameon=False)
plt.xlabel('$z_{tot}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
plt.axhline(y=1.0,color='black',linestyle='--')

plt.tight_layout()
plt.savefig('GiBUU_Momentum.png')

## Results vs GiBUU (Nu)

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='Nu',minz=2.4,maxz=5,nbins=5,pair_cut='and z_asso>0.1'
                       )
    plt.errorbar(x,r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8)

    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='Nu',minz=2.4,maxz=5,nbins=5,pair_cut='and z_asso>0.1'
                       )
    plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)

plt.legend(frameon=False)
plt.xlabel('Nu (GeV)',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
#plt.axhline(y=1.0,color='black',linestyle='--')

plt.tight_layout()
plt.savefig('GiBUU_Nu.png')

## Results vs GiBUU (Q2)

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['%s'%target], df['D_%s'%target], df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target]
                       ,variable='Q2',minz=1.0,maxz=4.0,nbins=8,pair_cut='and z_asso>0.1'
                       )
    plt.errorbar(x,r,yerr=err,label='This work, %s'%target,ms=8,fmt='o',alpha=0.8)

    r, err,x = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger']
                       ,variable='Q2',minz=1.0,maxz=4.0,nbins=8,pair_cut='and z_asso>0.1'
                       )
    plt.fill_between(x,r,r-err,r+err,label='GiBUU %s'%target,alpha=0.5)

plt.legend(frameon=False)
plt.xlabel('$Q^{2}$ (GeV$^{2}$)',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)
#plt.axhline(y=1.0,color='black',linestyle='--')

plt.tight_layout()
plt.savefig('GiBUU_Nu.png')

## R2h in mass bins

In [ ]:
fig, axs = plt.subplots(1, 3,sharex=True, sharey=True, figsize=(16,6), gridspec_kw={'hspace': 0, 'wspace':0})


mpl.rcParams['axes.labelsize'] = 17
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['errorbar.capsize'] = 5
mpl.rcParams['legend.fontsize']= 15

for target in ['C','Fe','Pb']:
    r, err,x   = getRatio(df[target], df['D_%s'%target], 
                          df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                          pair_cut='and mass<0.6 and mass>0.3')
    axs[0].errorbar(x,r,yerr=err,label='%s'%target)
axs[0].set_title('0.3< $M_{\pi\pi}$ <0.6 GeV',fontsize=16)
for target in ['C','Fe','Pb']:
    r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                 trig_cut='z>0.5', pair_cut='and mass<0.6 and mass>0.3')
    axs[0].errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=0,lw=1)

axs[0].axhline(y=1.0,color='black',linestyle='--')
axs[0].legend(frameon=False)
plt.xlim([0.0,0.5])

for target in ['C','Fe','Pb']:
    r, err,x   = getRatio(df[target], df['D_%s'%target], 
                          df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                          pair_cut='and mass>0.6 and mass<1.0')
    axs[1].errorbar(x,r,yerr=err,label='%s'%target)
axs[1].set_title('0.6< $M_{\pi\pi}$ <1.0 GeV',fontsize=16)
for target in ['C','Fe','Pb']:
    r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                 trig_cut='z>0.5', pair_cut='and mass>0.6 and mass<1.0')
    axs[1].errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=0,lw=1)

axs[1].axhline(y=1.0,color='black',linestyle='--')
axs[1].legend(frameon=False)
plt.xlim([0.0,0.5])


for target in ['C','Fe','Pb']:
    r, err,x   = getRatio(df[target], df['D_%s'%target], 
                          df_trigger['%s_trigger'%target], df_trigger['D_%s_trigger'%target],
                          pair_cut='and mass>1.0 and mass<1.5')
    axs[2].errorbar(x,r,yerr=err,label='%s'%target)
axs[2].set_title('1.0< $M_{\pi\pi}$ <1.5 GeV',fontsize=16)
for target in ['C','Fe']:
    r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                 trig_cut='z>0.5', pair_cut='and mass>1.0 and mass<1.5')
    axs[2].errorbar(x,r,yerr=err,label='GiBUU %s'%target,ms=0,lw=1)

axs[2].axhline(y=1.0,color='black',linestyle='--')
axs[2].legend(frameon=False)
plt.xlim([0.0,0.5])
plt.ylim([0.5,2.5])



axs[0].xaxis.set_label_text('$z_{2}$',fontsize=18)
axs[1].xaxis.set_label_text('$z_{2}$',fontsize=18)
axs[2].xaxis.set_label_text('$z_{2}$',fontsize=18)

axs[0].yaxis.set_label_text('$R_{2h}$',fontsize=18)

plt.tight_layout()
plt.savefig('R2h_massdependence.png')
plt.show()

## Compare results with and without z_tot<0.8 cut

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(12,6), gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['C','Fe','Pb']:
    r, err, x = getRatio(df[target],df['D_%s'%target],df_trigger['%s_trigger'%target],df_trigger['D_%s_trigger'%target],nbins=24,variable='z_asso')
    axs[0].errorbar(x,r,yerr=err,label=target,fmt='o',ms=8,lw=3,capsize=5)
    r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                          )
    axs[0].fill_between(x,r+err,r-err,label='GiBUU %s'%target,alpha=0.5)

    r, err, x = getRatio(df[target],df['D_%s'%target],df_trigger['%s_trigger'%target],df_trigger['D_%s_trigger'%target],nbins=24,variable='z_asso',pair_cut=' and z_tot<0.8')
    axs[1].errorbar(x,r,yerr=err,label=target,fmt='o',mfc='white',ms=8,lw=3,capsize=5)
    r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],df_trigger['GiBUU_D_trigger'],
                         pair_cut=' and z_tot<0.80')
    axs[1].fill_between(x,r+err,r-err,label='GiBUU %s'%target,alpha=0.5)
    
    
axs[1].axhline(y=1.0,color='black',linestyle='--')
axs[0].axhline(y=1.0,color='black',linestyle='--')

axs[1].set_title('with $z_1+z_2<$0.8', fontsize=13)
axs[0].set_title('no cut ', fontsize=13)

axs[0].legend()
axs[1].legend()

plt.tight_layout()
plt.savefig('ExclusiveCutVariation.png')
plt.show()

## Checking the dependency of the ratios on varios things like the charged of leading pion, subleading pion; Q2 and Nu

In [ ]:

for target in ['C','Fe','Pb']:
    fig, axs = plt.subplots(1, 2, sharey=True, figsize=(12,6), gridspec_kw={'hspace': 0, 'wspace':0})
  
    printPairBreakdown(df['%s'%target])

    ##Negative associated hadron
    r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
                          df_trigger['D_%s_trigger'%target],trig_cut='pid>0 and z<0.7', pair_cut='and pid_asso>0',maxz=0.4)
    axs[0].errorbar(x,r,yerr=err,label='%s, +/+'%target,fmt='o',ms=8,lw=3,capsize=5)
    
    r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
                          df_trigger['D_%s_trigger'%target], trig_cut = 'pid<0 and z<0.7',pair_cut='and pid_asso>0',maxz=0.4)
    axs[0].errorbar(x,r,yerr=err,label='%s, -/+'%target,fmt='o',ms=8,lw=3,capsize=5)
    
    
    r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
                          df_trigger['D_%s_trigger'%target],trig_cut='pid>0 and z<0.7', pair_cut='and pid_asso<0',maxz=0.4)
    axs[0].errorbar(x,r,yerr=err,label='%s, +/-'%target,fmt='o',ms=8,lw=3,capsize=5)
    
    #r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
    #                      df_trigger['D_%s_trigger'%target], trig_cut = 'pid<0',pair_cut='and pid_asso<0')
    #axs[0].errorbar(x,r,yerr=err,label='%s, -/-'%target,fmt='o',ms=8,lw=3,capsize=5)
    
    
    ## no charge cut
    #r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
    #                      df_trigger['D_%s_trigger'%target], minz=0.07)
    #axs[0].errorbar(x,r,yerr=err,label='%s, all'%target,fmt='o',ms=8,lw=3,capsize=5)  


    ##GiBUU negative trigger
    #r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],
    #                      df_trigger['GiBUU_D_trigger'], pair_cut='and pid_asso<0')                                              
    #axs[1].errorbar(x,r,yerr=err,label='GiBUU %s, pi- asso'%target,ms=8,lw=3,capsize=5)
    
    ##GiBUU positive trigger
    #r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],
    #                      df_trigger['GiBUU_D_trigger'], pair_cut='and pid_asso>0')                                              
    #axs[1].errorbar(x,r,yerr=err,label='GiBUU %s, pi+ asso'%target,ms=8,lw=3,capsize=5)
    
    ##GiBUU all hadrons
    #r, err, x  = getRatio(df['GiBUU_%s'%target],df['GiBUU_D'],df_trigger['GiBUU_%s_trigger'%target],
    #                      df_trigger['GiBUU_D_trigger'])                                              
    #axs[1].errorbar(x,r,yerr=err,label='GiBUU %s, all'%target,ms=8,lw=3,capsize=5)
    
    plt.ylim([0.0,1.0])


    axs[1].axhline(y=1.0,color='black',linestyle='--')
    axs[0].axhline(y=1.0,color='black',linestyle='--')

    axs[0].legend(frameon=False)
    axs[1].legend(frameon=False)


    plt.savefig('ConsistencyCharge_%s.png'%target)
    plt.show()
    plt.clf()

## R_2h as a function of theta

In [ ]:
plt.hist(df['C'].query('pid>0')['theta_lab_asso'],bins=100,alpha=0.5,label='positive trigger')
plt.hist(df['C'].query('pid<0')['theta_lab_asso'],bins=100,alpha=0.5,label='negative trigger')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(12,6), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [25,40,55,80]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]
for target in ['Fe']:
    for j, ibin in enumerate(bins):

        pair_cut = 'and theta_lab_asso>%2.2f and theta_lab_asso<=%2.2f'%(ibin[0],ibin[1])
        label = '%2.0f$<\Theta<$%2.0f, D'%(ibin[0],ibin[1])

    
    ##Positive trigger
        r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
                          df_trigger['D_%s_trigger'%target],trig_cut='pid>0', 
                          pair_cut=pair_cut,minz=0.07,nbins=8)
        axs[0].errorbar(x,r,yerr=err,fmt='o',ms=8,lw=3,capsize=5,label=label)
        
        ##negative trigger
        
        r, err, x  = getRatio(df['%s'%target],df['D_%s'%target],df_trigger['%s_trigger'%target],
                          df_trigger['D_%s_trigger'%target],trig_cut='pid<0', 
                          pair_cut=pair_cut,minz=0.07,nbins=8)
        axs[1].errorbar(x,r,yerr=err,fmt='o',ms=8,lw=3,capsize=5,label=label)
        
axs[0].legend(frameon=False,fontsize=12,ncol=2)
axs[1].legend(frameon=False,fontsize=12,ncol=1)    




## z2 distribution breakdown for pi+ and pi- for all nuclei

In [ ]:
fig = plt.figure( figsize=(12, 10))

for i, target in enumerate(['D','C','Fe','Pb']):
    plt.subplot(2,2,i+1)
    plt.hist(df[target].query('z>0.5 and pid_asso>0')['z_asso'], bins=100,range =(0,0.5),label='$\pi^+$',alpha=0.5)
    plt.hist(df[target].query('z>0.5 and pid_asso<0')['z_asso'], bins=100,range =(0,0.5),label='$\pi^-$',alpha=0.5)

    plt.title('%s, Conditional for $z_{1}>0.5$'%target)
    plt.ylabel('pairs',fontsize=16)
    plt.xlabel('$z_{2}$',fontsize=16)
    plt.legend(frameon=False)    

plt.savefig('ConditionalDistribution_z2_BothPions.png')

# z distribution of triggers by charge

In [ ]:
fig = plt.figure( figsize=(12, 10))

for i, target in enumerate(['D','C','Fe','Pb']):
    plt.subplot(2,2,i+1)
    plt.hist(df_trigger['%s_trigger'%target].query('z>0.5 and pid>0')['z'], bins=100,range =(0.5,1.0),label='$\pi^+$',alpha=0.5)
    plt.hist(df_trigger['%s_trigger'%target].query('z>0.5 and pid<0')['z'], bins=100,range =(0.5,1.0),label='$\pi^-$',alpha=0.5)

    plt.ylabel('pairs',fontsize=16)
    plt.xlabel('$z_{1}$',fontsize=16)
    plt.legend(frameon=False)    

plt.savefig('Triggers.png')

## Distributions


In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,6),sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['D','C','Fe','Pb']:
    axs[0].hist(df['GiBUU_%s_trigger'%target]['z_asso'],bins=50,density=True,range=(0.0,0.5),lw=2,label='GiBUU, %s'%target,histtype='step')
    axs[1].hist(df['%s_trigger'%target]['z_asso'],bins=50,density=True,range=(0.0,0.5),lw=2,label=' %s'%target,histtype='step')

axs[0].legend(frameon=False)
axs[1].legend(frameon=False)

axs[0].xaxis.set_label_text('$z_{2}$',fontsize=15)
axs[1].xaxis.set_label_text('$z_{2}$',fontsize=15)

axs[0].yaxis.set_label_text("normalized units", fontsize=15)
axs[0].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)
axs[1].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,6),sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['D','C','Fe','Pb']:
    axs[0].hist(df['GiBUU_%s'%target]['P_asso'],bins=50,density=True,range=(0.0,2.0),lw=2,label='GiBUU, %s'%target,histtype='step')
    axs[1].hist(df['%s'%target]['P_asso'],bins=50,density=True,range=(0.0,2.0),lw=2,label=' %s'%target,histtype='step')

axs[0].legend(frameon=False)
axs[1].legend(frameon=False)

axs[0].xaxis.set_label_text('$P_{2}$ (GeV)',fontsize=18)
axs[1].xaxis.set_label_text('$P_{2}$ (GeV)',fontsize=18)

axs[0].yaxis.set_label_text("normalized units", fontsize=18)
axs[0].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)
axs[1].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,6),sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['D','C','Fe','Pb']:
    axs[0].hist(df_trigger['GiBUU_%s_trigger'%target]['Nu'],bins=50,density=True,range=(2.0,5.0),lw=2,label='GiBUU, %s'%target,histtype='step')
    axs[1].hist(df_trigger['%s_trigger'%target]['Nu'],bins=50,density=True,range=(2.0,5.0),lw=2,label=' %s'%target,histtype='step')

axs[0].legend(frameon=False)
axs[1].legend(frameon=False)

axs[0].xaxis.set_label_text('Nu (GeV)',fontsize=18)
axs[1].xaxis.set_label_text('Nu (GeV)',fontsize=18)

axs[0].yaxis.set_label_text("normalized units", fontsize=18)
#axs[0].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)
#axs[1].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,6),sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['D','C','Fe','Pb']:
    axs[0].hist(df_trigger['GiBUU_%s_trigger'%target]['Q2'],bins=50,density=True,range=(1.0,4.0),lw=2,label='GiBUU, %s'%target,histtype='step')
    axs[1].hist(df_trigger['%s_trigger'%target]['Q2'],bins=50,density=True,range=(1.0,4.0),lw=2,label=' %s'%target,histtype='step')

axs[0].legend(frameon=False)
axs[1].legend(frameon=False)

axs[0].xaxis.set_label_text('Q2 (GeV2)',fontsize=18)
axs[1].xaxis.set_label_text('Q2 (GeV2)',fontsize=18)

axs[0].yaxis.set_label_text("normalized units", fontsize=18)
#axs[0].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)
#axs[1].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,6),sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['D','C','Fe','Pb']:
    axs[0].hist(df['GiBUU_%s'%target]['z_tot'],bins=50,density=True,range=(0.5,1.0),lw=2,label='GiBUU, %s'%target,histtype='step')
    axs[1].hist(df['%s'%target]['z_tot'],bins=50,density=True,range=(0.5,1.0),lw=2,label=' %s'%target,histtype='step')

axs[0].legend(frameon=False)
axs[1].legend(frameon=False)

axs[0].xaxis.set_label_text('$z_{tot}$',fontsize=18)
axs[1].xaxis.set_label_text('$z_{tot}$',fontsize=18)

axs[0].yaxis.set_label_text("normalized units", fontsize=18)
axs[0].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)
axs[1].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,6),sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

for target in ['D','C','Fe','Pb']:
    axs[0].hist(df['GiBUU_%s'%target]['mass'],bins=50,density=True,range=(0.0,2.0),lw=2,label='GiBUU, %s'%target,histtype='step')
    axs[1].hist(df['%s'%target]['mass'],bins=50,density=True,range=(0.0,2.0),lw=2,label=' %s'%target,histtype='step')

axs[0].legend(frameon=False)
axs[1].legend(frameon=False)

axs[0].xaxis.set_label_text('Dipion mass (GeV)',fontsize=18)
axs[1].xaxis.set_label_text('Dipion mass (GeV)',fontsize=18)

axs[0].yaxis.set_label_text("normalized units", fontsize=18)
axs[0].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)
axs[1].set_title('Events with a leading pion $z_{1}>0.5$',fontsize=15)

plt.show()

In [ ]:
for target in ['C','Fe','Pb']:
    y, x = np.histogram(df_trigger['%s_trigger'%target]['z'], bins=100,range =(0.5,1.0))
    y_D, x = np.histogram(df_trigger['D_%s_trigger'%target]['z'], bins=100,range =(0.5,1.0))
    x = (x[:-1] + x[1:])/2.0
    ratio = np.true_divide(y,y_D)
    plt.plot(x,ratio,label=target)

plt.xlabel('z')
plt.ylabel('ratio to deuterium')
plt.legend(frameon=False)
plt.show()
plt.savefig('Triggers_z1distributions_RatioToD.png')

In [ ]:
fig = plt.figure( figsize=(18, 4))

for i, target in enumerate(['C','Fe','Pb']):
    plt.subplot(1,3,i+1)
    plt.hist(df_trigger['%s_trigger'%target]['z'], bins=100,range =(0.5,1.0),histtype='step', label=target)
    plt.hist(df_trigger['D_%s_trigger'%target]['z'], bins=100,range =(0.5,1.0),histtype='step',label='D (%s)'%target)

    plt.ylabel('Number of pions',fontsize=13)
    plt.xlabel('$z_{1}$',fontsize=13)
    plt.legend(frameon=False)    

plt.savefig('Triggers_z1distributions.png')

In [ ]:
plt.hexbin(df_trigger['D_trigger']['Q2'],df_trigger['D_trigger']['Nu'], cmap='inferno')


In [ ]:
plt.hexbin(df_trigger['GiBUU_D_trigger']['Q2'],df_trigger['GiBUU_D_trigger']['Nu'],cmap='inferno')
